In [ ]:
import numpy as np
import numba as nb
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
from numba import njit, prange
from os import listdir
from os.path import isfile, join

%matplotlib inline
plt.style.use(['science','ieee']) # Ini kudu install install dulu

# IMPORT THE TRAINING DATA

In [ ]:
for i in range(0,24):
    if i < 12:
        df = pd.read_csv('../data/training_data/' + str(i) + '.csv')
    else:
        df = pd.read_csv('../data/training_data/s' + str(i-12) + '.csv')
    throttle = df.throttle
    brake = df.brake
    exec("t{} = np.array(df.t).astype('float64')".format(i))
    exec("u{} = np.array([throttle, brake]).T.astype('float64')".format(i))
    exec("v{} = np.array(df.v).astype('float64')".format(i))

# MAKE FUNCTION

## Generate population

In [ ]:
def generate_population(num, dim, rng):
    """
    Generate population:
        Input:
            num: number of population (integer)
            dim: number of parameters (integer)
            rng: range number used in initialization (list or numpy array)
        Output:
            pop: initial position of the population (numpy array)
    """
    pop = np.zeros((num,dim))
    for i in range(dim):
        lim = rng[i]
        pop[:, i] = np.random.uniform(lim[0], lim[1], size=num)
    return pop

## Constraint

In [ ]:
ndim = 17
@njit
def admissible(param):
    a1, a2, a3, b1, b2, b3, b4, c1, c2, c3, c4, td11, td12, td13, td21, td22, td23 = param
    if a1 > 0. or a2 > 0. or a3 > 0. or b1 < 0. or b2 < 0. or c1 > 0. or c2 > 0. or np.around(td11)<0 or np.around(td12)<0 or np.around(td13)<0 or np.around(td21)<0 or np.around(td22)<0 or np.around(td23)<0:
        return False
    else:
        return True
    
_ = admissible(np.random.randn(ndim))

## Forward propagation

In [ ]:
@njit
def delayed_control_signal(i, u, u_list, td):
    if i < td:
        ut = 0.0
    else:
        if td == 0:
            ut = u
        else:
            ut = u_list[i-td]
    return ut
_ = delayed_control_signal(1, 0.1, np.array([0.1, 0.2]), 0)

In [ ]:
@njit
def forward_propagation(t, u, param):
    a1, a2, a3, b1, b2, b3, b4, c1, c2, c3, c4, td11, td12, td13, td21, td22, td23 = param
    td11 = int(np.around(td11))
    td12 = int(np.around(td12))
    td13 = int(np.around(td13))
    td21 = int(np.around(td21))
    td22 = int(np.around(td22))
    td23 = int(np.around(td23))

    u1_list = np.empty(t.shape)
    u2_list = np.empty(t.shape)
    out = np.empty(t.shape)
    y = 0.0
    for i in range(t.shape[0]):
        if i == 0:
            dt = t[1] - t[0]
        else:
            dt = t[i] - t[i-1]
            
        u1, u2 = u[i]        
        u11t = delayed_control_signal(i, u1, u1_list, td11)
        u12t = delayed_control_signal(i, u1, u1_list, td12)
        u13t = delayed_control_signal(i, u1, u1_list, td13)
        u21t = delayed_control_signal(i, u2, u2_list, td21)
        u22t = delayed_control_signal(i, u2, u2_list, td22)
        u23t = delayed_control_signal(i, u2, u2_list, td23)
        
        temp = 0.
        if y != 0.:
            temp = a1
                    
        y_dot = temp + a2 * y + a3 * y**2 \
                + b1 * u11t + b2 * np.exp(b3 * y + b4 * u12t) * u13t \
                + c1 * u21t + c2 * np.exp(c3 * y + c4 * u22t) * u23t
        
        y += y_dot * dt
        if y < 0.0:
            y = 0.0

        u1_list[i] = u1
        u2_list[i] = u2
        out[i] = y

    return out
_ = forward_propagation(np.arange(10, dtype=float), np.random.randn(10,2), np.ones(ndim))

## Cost and metric

In [ ]:
@njit
def cost(t, u, v, param):
    error = (v - forward_propagation(t, u, param))
    cost = np.dot(error.T, error)
    return cost
_ = cost(np.arange(10, dtype=float), np.random.randn(10,2), np.random.randn(10), np.ones(ndim))

@njit
def accuracy(t, u, v, param):
    error = v - forward_propagation(t, u, param)
    numerator = np.linalg.norm(error)
    denumerator = np.linalg.norm(v - np.mean(v))
    acc = 1.0 - numerator / denumerator
    return acc
_ = accuracy(np.arange(10, dtype=float), np.random.randn(10,2), np.random.randn(10), np.ones(ndim))

@njit
def mean_absolute_error(t, u, v, param):
    error = v - forward_propagation(t, u, param)
    out = np.mean(np.abs(error))
    return out
_ = mean_absolute_error(np.arange(10, dtype=float), np.random.randn(10,2), np.random.randn(10), np.ones(ndim))

@njit
def mean_squared_error(t, u, v, param):
    error = v - forward_propagation(t, u, param)
    out = np.mean(error**2)
    return out
_ = mean_squared_error(np.arange(10, dtype=float), np.random.randn(10,2), np.random.randn(10), np.ones(ndim))

@njit
def maximum_absolute_error(t, u, v, param):
    error = v - forward_propagation(t, u, param)
    out = np.max(np.abs(error))
    return out
_ = maximum_absolute_error(np.arange(10, dtype=float), np.random.randn(10,2), np.random.randn(10), np.ones(ndim))

In [ ]:
for i in range(0,24):
    if i < 12:
        df = pd.read_csv('../data/training_data/' + str(i) + '.csv')
    else:
        df = pd.read_csv('../data/training_data/s' + str(i-12) + '.csv')
    throttle = df.throttle
    brake = df.brake
    exec("t{} = np.array(df.t).astype('float64')".format(i))
    exec("u{} = np.array([throttle, brake]).T.astype('float64')".format(i))
    exec("v{} = np.array(df.v).astype('float64')".format(i))

In [ ]:
gain = np.array([1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0,
                 1.0, 1.0, 1.0])

@njit
def calculate_total_cost(param):
    loss = np.Inf
    if admissible(param):
        loss = 0.
        m = 0.
        
        loss += cost(t0, u0, v0, param) * gain[0]
        loss += cost(t1, u1, v1, param) * gain[1]
        loss += cost(t2, u2, v2, param) * gain[2]
        loss += cost(t3, u3, v3, param) * gain[3]
        loss += cost(t4, u4, v4, param) * gain[4]
        loss += cost(t5, u5, v5, param) * gain[5]
        loss += cost(t6, u6, v6, param) * gain[6]
        loss += cost(t7, u7, v7, param) * gain[7]
        loss += cost(t8, u8, v8, param) * gain[8]
        loss += cost(t9, u9, v9, param) * gain[9]
        loss += cost(t10, u10, v10, param) * gain[10]
        loss += cost(t11, u11, v11, param) * gain[11]
        loss += cost(t12, u12, v12, param) * gain[12]
        loss += cost(t13, u13, v13, param) * gain[13]
        loss += cost(t14, u14, v14, param) * gain[14]
        loss += cost(t15, u15, v15, param) * gain[15]
        loss += cost(t16, u16, v16, param) * gain[16]
        loss += cost(t17, u17, v17, param) * gain[17]
        loss += cost(t18, u18, v18, param) * gain[18]
        loss += cost(t19, u19, v19, param) * gain[19]
        loss += cost(t20, u20, v20, param) * gain[20]
        loss += cost(t21, u21, v21, param) * gain[21]
        loss += cost(t22, u22, v22, param) * gain[22]
        loss += cost(t23, u23, v23, param) * gain[23]

        m += t0.shape[0] * gain[0]
        m += t1.shape[0] * gain[1]
        m += t2.shape[0] * gain[2]
        m += t3.shape[0] * gain[3]
        m += t4.shape[0] * gain[4]
        m += t5.shape[0] * gain[5]
        m += t6.shape[0] * gain[6]
        m += t7.shape[0] * gain[7]
        m += t8.shape[0] * gain[8]
        m += t9.shape[0] * gain[9]
        m += t10.shape[0] * gain[10]
        m += t11.shape[0] * gain[11]
        m += t12.shape[0] * gain[12]
        m += t13.shape[0] * gain[13]
        m += t14.shape[0] * gain[14]
        m += t15.shape[0] * gain[15]
        m += t16.shape[0] * gain[16]
        m += t17.shape[0] * gain[17]
        m += t18.shape[0] * gain[18]
        m += t19.shape[0] * gain[19]
        m += t20.shape[0] * gain[20]
        m += t21.shape[0] * gain[21]
        m += t22.shape[0] * gain[22]
        m += t23.shape[0] * gain[23]

        loss = loss / m
        
    return loss
_ = calculate_total_cost(np.ones(ndim))

In [ ]:
@njit#(parallel=True)
def population_cost(population):
    length = population.shape[0]
    losses = np.empty(length)
    #for ii in prange(length):
    for ii in range(length):
        losses[ii] = calculate_total_cost(population[ii])
    return losses
_ = population_cost(np.ones((2,ndim)))

## PSO

In [ ]:
@njit
def pso(population, pop_velocity, global_, global_loss, best_pos_local,
        best_local_loss, w_max, w_min, c1, c2, iteration, max_iter):
    num = population.shape[0]
    dim = population.shape[1]
    
    # Initial conditions
    ppos_vector = np.copy(population)
    pvel_vector = np.copy(pop_velocity)
    gbest_pos = np.copy(global_)
    gfit_value = global_loss + 0.0 # to avoid shallow copy
    pbest_pos = np.copy(best_pos_local)
    pfit_value = np.copy(best_local_loss)

    # Additional variables for catching the next global best
    next_gbest_pos = np.copy(gbest_pos)
    next_gfit_value = gfit_value + 0.0 # to avoid shallow copy
    
    # Calculate w based on w_max, w_min, current number of iteration,
    # and the maximum iteration
    w = w_max - (w_max - w_min) * iteration / max_iter
    
    for i in range(num):        
        # Get random numbers for r1 and r2
        r1 = np.random.uniform(0., 1.)
        r2 = np.random.uniform(0., 1.)
        
        # Update the velocity and position vector
        d1 = pbest_pos[i] - ppos_vector[i]
        d2 = gbest_pos - ppos_vector[i]
        pvel_vector[i] = w*pvel_vector[i] + c1*r1*d1+ c2*r2*d2
        ppos_vector[i] = pvel_vector[i] + ppos_vector[i]
        
        # Calculate the cost function or fit value
        cost_func = calculate_total_cost(ppos_vector[i])
        
        # Update the next local best for i-th particle if applicable
        if(pfit_value[i] > cost_func):
            pfit_value[i] = cost_func + 0.0 # to avoid shallow copy
            pbest_pos[i] = np.copy(ppos_vector[i])
                
        # Update the next global best if applicable
        if(next_gfit_value > cost_func):
            next_gfit_value = cost_func  + 0.0 # to avoid shallow copy
            next_gbest_pos = np.copy(ppos_vector[i])
        
    return ppos_vector, pvel_vector, pbest_pos, pfit_value, next_gbest_pos, next_gfit_value

xx1 = np.ones((2, ndim))
xx2 = np.ones(2)
xx3 = np.random.randn(ndim)
_ = pso(xx1, xx1, xx3, 100.0, xx1*0.5, xx2, 0.9, 0.4, 0.7, 0.8, 0, 20)

# SIMULATION (OPTIMIZATION)

In [ ]:
num = 50
n_sim = 20
n_itr = 10000

r_a1 = [0., -2.]
r_a2 = [0., -2.]
r_a3 = [0., -2.]
r_b1 = [0., 2.]
r_b2 = [0., 2.]
r_b3 = [-2., 2.]
r_b4 = [-2., 2.]
r_c1 = [0., -2.]
r_c2 = [0., -2.]
r_c3 = [-2., 2.]
r_c4 = [-2., 2.]
r_td11 = [0, 15]
r_td12 = [0, 15]
r_td13 = [0, 15]
r_td21 = [0, 6]
r_td22 = [0, 6]
r_td23 = [0, 6]
rng = [r_a1, r_a2, r_a3,
       r_b1, r_b2, r_b3, r_b4,
       r_c1, r_c2, r_c3, r_c4,
       r_td11, r_td12, r_td13,
       r_td21, r_td22, r_td23
      ]
dim = len(rng)

w_max = 0.9
w_min = 0.4
c1 = 0.7
c2 = 0.8

In [ ]:
param_history = np.zeros((n_sim, dim))
loss_history = np.ones(n_sim)*np.Inf

the_best_param_history = np.zeros((n_itr, dim))
the_best_loss_history = np.zeros(n_itr)

for j in range(n_sim):
    population = generate_population(num, dim, rng)
    rng_vel = [[-max([abs(rng[i][0]), abs(rng[i][1])]), max([abs(rng[i][0]), abs(rng[i][1])])] for i in range(len(rng))]
    pop_velocity = generate_population(num, dim, rng_vel)
    global_ = None
    global_loss_ = np.Inf
    local_best_pos = np.copy(population)
    
    loss_population = population_cost(population)
    loss_population[np.isnan(loss_population)] = np.Inf
    min_idx = np.argmin(loss_population)
    min_loss = loss_population[min_idx]
    if global_loss_ > min_loss:
        global_loss_ = min_loss
        global_ = np.copy(population[min_idx, :])
        
    global_history = np.empty((n_itr, dim))
    global_history[0] = global_
    global_loss_history = np.empty(n_itr)
    global_loss_history[0] = global_loss_
    for i in range(1, n_itr):
        # Particle Swarm Optimization
        population, pop_velocity, local_best_pos, loss_population, global_, global_loss_ = pso(
            population, pop_velocity, global_, global_loss_, local_best_pos, loss_population,
            w_max, w_min, c1, c2, i, n_itr
        )
        
        global_history[i] = global_
        global_loss_history[i] = global_loss_
    
    if np.min(loss_history) > global_loss_history[-1]:
        the_best_loss_history = np.copy(global_loss_history)
        the_best_param_history = np.copy(global_history)
        
    param_history[j] = np.copy(global_history[-1])
    loss_history[j] = np.copy(global_loss_history[-1])
    
    print('simulation: {} || global loss: {:.10f} || the best loss: {:.10f}'.format(j, global_loss_history[-1], the_best_loss_history[-1]))

In [ ]:
# Save the simulation
np.save('result/param_history.npy', param_history)
np.save('result/loss_history.npy', loss_history)
np.save('result/the_best_loss_history.npy', the_best_loss_history)
np.save('result/the_best_param_history.npy', the_best_param_history)

In [ ]:
f = open("result/sim.cfg", "w+")
f.writelines('num: {} # The number of particles\n'.format(num))
f.writelines('n_sim: {} # The number of simulation loop\n'.format(n_sim))
f.writelines('n_itr: {} # The number of iteration for each simulation\n'.format(n_itr))
f.writelines('\n# The boundary of the initialization value\n')
f.writelines('\tr_a1: {}\n'.format(r_a1))
f.writelines('\tr_a2: {}\n'.format(r_a2))
f.writelines('\tr_a3: {}\n'.format(r_a3))
f.writelines('\tr_b1: {}\n'.format(r_b1))
f.writelines('\tr_b2: {}\n'.format(r_b2))
f.writelines('\tr_b3: {}\n'.format(r_b3))
f.writelines('\tr_b4: {}\n'.format(r_b4))
f.writelines('\tr_c1: {}\n'.format(r_c1))
f.writelines('\tr_c2: {}\n'.format(r_c2))
f.writelines('\tr_c3: {}\n'.format(r_c3))
f.writelines('\tr_c4: {}\n'.format(r_c4))
f.writelines('\tr_td11: {}\n'.format(r_td11))
f.writelines('\tr_td12: {}\n'.format(r_td12))
f.writelines('\tr_td13: {}\n'.format(r_td13))
f.writelines('\tr_td21: {}\n'.format(r_td21))
f.writelines('\tr_td22: {}\n'.format(r_td22))
f.writelines('\tr_td23: {}\n'.format(r_td23))
f.writelines('\n# The gain of the dataset\n')
for i in range(gain.shape[0]):
    f.writelines('\tdata-{}: {}\n'.format(i, gain[i]))
f.writelines('\n# The PSO hyperparameters\n')
f.writelines('w: {}\n'.format(w))
f.writelines('c1: {}\n'.format(c1))
f.writelines('c2: {}\n'.format(c2))
f.close()

# RESULT

## Load Model

In [ ]:
# Load the model
the_best_param_history = np.load('result/the_best_param_history.npy')
the_best_loss_history = np.load('result/the_best_loss_history.npy')
global_ = the_best_param_history[-1]
global_loss_ = the_best_loss_history[-1]

In [ ]:
param_name = ['a1', 'a2', 'a3', 'b1', 'b2', 'b3', 'b4', 'c1', 'c2', 'c3', 'c4', 'd11', 'd12', 'd13', 'd21', 'd22', 'd23']
print('The minimum loss: {}'.format(global_loss_))
print('Parameters:')
for i in range(len(param_name)):
    print('{}: {}'.format(param_name[i], global_[i]))

## Visualization

In [ ]:
# Visualization
idx = 12
exec("t = np.copy(t{})".format(idx))
exec("u = np.copy(u{})".format(idx))
exec("v = np.copy(v{})".format(idx))
test = forward_propagation(t, u, global_)
plt.plot(t, v, label='ground-truth')
plt.plot(t, test, label='prediction')
plt.legend()
plt.show()

## Cost and Metrics

In [ ]:
acc = []
mse = []
maae = [] 
for i in range(24):
    exec("acc.append(accuracy(t{}, u{}, v{}, global_))".format(i, i, i))
    exec("mse.append(mean_squared_error(t{}, u{}, v{}, global_))".format(i, i, i))
    exec("maae.append(maximum_absolute_error(t{}, u{}, v{}, global_))".format(i, i, i))

In [ ]:
print('Accuracy')
for i in range(24):
    print('Data ke-{}: {:.2f}%'.format(i+1, acc[i]*100))

In [ ]:
print('Mean Squared Error')
for i in range(24):
    print('Data ke-{}: {:.5f}'.format(i+1, mse[i]))

In [ ]:
print('Maximum Absolute Squared Error')
for i in range(24):
    print('Data ke-{}: {:.3f}'.format(i+1, maae[i]))

In [ ]:
num = 0
for i in range(24):
    exec("num += t{}.shape[0]".format(i))
vv = np.empty(num)
vv_gt = np.empty(num)
nn = 0
for i in range(24):
    exec("n = t{}.shape[0]".format(i))
#     if nn == 0:
#         exec("vv[:n] = forward_propagation(t{}, u{}, global_)".format(i, i))
#         exec("vv_gt[:n] = v{}".format(i))
#     else:
#         exec("vv[nn: nn+n] = forward_propagation(t{}, u{}, global_)".format(i, i))
#         exec("vv_gt[nn:nn+n] = v{}".format(i))
    exec("vv[nn: nn+n] = forward_propagation(t{}, u{}, global_)".format(i, i))
    exec("vv_gt[nn: nn+n] = v{}".format(i))
    nn += n
total_accuracy = (1 - np.linalg.norm(vv_gt - vv)/np.linalg.norm(vv_gt - np.mean(vv_gt)))
print('Total Accuracy: {:.2f}%'.format(total_accuracy*100))

# TEST MODEL

In [ ]:
df = pd.read_csv('test_data.csv')
throttle = df.throttle
brake = df.brake
t99 = np.array(df.time).astype('float64')
u99 = np.array([throttle, brake]).T.astype('float64')
v99 = np.array(df.speed).astype('float64')

In [ ]:
vp = forward_propagation(t99, u99, global_)
print('accuracy: {:.2f}%'.format(100*accuracy(t99, u99, v99, global_)))
print('mean_squared_error: {:.5f}'.format(mean_squared_error(t99, u99, v99, global_)))
print('max_absolute_error: {:.3f}'.format(maximum_absolute_error(t99, u99, v99, global_)))

In [ ]:
plt.plot(t99, v99, label='Real Speed', linestyle='--', color='black', linewidth=2)
plt.plot(t99, vp, label='Estimated', linewidth=1, color='red')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Speed (m/s)')
plt.title('Real Speed vs Estimated Speed')
plt.savefig('results/real_estimated', dpi=600)
plt.show()

In [ ]:
plt.plot(t99, v99-vp, linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Error (m/s)')
plt.title('Estimation Error')
plt.savefig('results/estimation_error', dpi=600)
plt.show()

In [ ]:
err = v99 - vp
plt.hist(v99-vp, bins=int((np.max(err) - np.min(err))/0.01))
plt.title('Histogram of Estimation Error')
plt.xlabel('Error (m/s)')
plt.ylabel('Frequency')
plt.savefig('results/estimation_error_hist', dpi=600)
plt.show()

## Dari Data CARLA dengan PID

In [ ]:
df = pd.read_csv('test_data_pid.csv')
throttle = df.throttle
brake = df.brake
mt0 = np.array(df.time).astype('float64')
mu0 = np.array([throttle, brake]).T.astype('float64')
mv0 = np.array(df.speed).astype('float64')

In [ ]:
global_ = global_history[-1]
vp0 = forward_propagation(mt0, mu0, global_)
print('accuracy: {:.2f}%'.format(100*accuracy(mt0, mu0, mv0, global_)))
print('mean_squared_error: {:.5f}'.format(mean_squared_error(mt0, mu0, mv0, global_)))
print('max_absolute_error: {:.3f}'.format(maximum_absolute_error(mt0, mu0, mv0, global_)))

In [ ]:
plt.plot(mt0, mv0, label='Real Speed', linestyle='--', color='black', linewidth=2)
plt.plot(mt0, vp0, label='Estimated', linewidth=1, color='red')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Speed (m/s)')
plt.title('Real Speed vs Estimated Speed')
plt.savefig('results/real_estimated_II', dpi=600)
plt.show()

In [ ]:
plt.plot(mt0, mv0-vp0, linewidth=2)
plt.xlabel('Time (s)')
plt.ylabel('Error (m/s)')
plt.title('Estimation Error')
plt.savefig('results/estimation_error_II', dpi=600)
plt.show()

In [ ]:
err = mv0-vp0
plt.hist(mv0-vp0, bins=int((np.max(err) - np.min(err))/0.01))
plt.title('Histogram of Estimation Error')
plt.xlabel('Error (m/s)')
plt.ylabel('Frequency')
plt.savefig('results/estimation_error_hist_II', dpi=600)
plt.show()